## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-28-17-02-00 +0000,wsj,Two Federal Immigration Officials Fired Shots ...,https://www.wsj.com/us-news/two-federal-immigr...
1,2026-01-28-17-01-00 +0000,startribune,The latest: Rep. Ilhan Omar sprayed with unkno...,https://www.startribune.com/ice-raids-minnesot...
2,2026-01-28-17-00-26 +0000,bbc,Starmer vows to push ahead with cuts to jury t...,https://www.bbc.com/news/articles/cp87pl83nvko...
3,2026-01-28-17-00-10 +0000,bbc,"Chagos discussions with US continue, says Starmer",https://www.bbc.com/news/articles/cp80p875pdko...
4,2026-01-28-16-59-02 +0000,bbc,Panic on crowded train - passenger tells BBC o...,https://www.bbc.com/news/articles/cgjwpq4pggjo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2446/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,64
315,ice,21
96,minneapolis,21
148,minnesota,16
1,immigration,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
198,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,144
226,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...,132
14,2026-01-28-16-34-43 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,132
98,2026-01-28-13-05-30 +0000,bbc,Trump says government will 'de-escalate' in Mi...,https://www.bbc.com/news/articles/c0q425vg4qzo...,127
110,2026-01-28-12-30-42 +0000,nypost,"Trump calls GOP Sens. Murkowski, Tillis ‘loser...",https://nypost.com/2026/01/28/us-news/gop-sens...,125


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
198,144,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
230,86,2026-01-28-01-32-14 +0000,cbc,2 federal agents fired shots when Alex Pretti ...,https://www.cbc.ca/news/world/alex-pretti-shoo...
226,68,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...
303,60,2026-01-27-21-52-09 +0000,nypost,Trump’s ‘big armada’ with aircraft carrier Abe...,https://nypost.com/2026/01/27/world-news/trump...
379,56,2026-01-27-17-53-30 +0000,nypost,TikTok’s new owners accused of censoring Minne...,https://nypost.com/2026/01/27/business/tiktok-...
1,55,2026-01-28-17-01-00 +0000,startribune,The latest: Rep. Ilhan Omar sprayed with unkno...,https://www.startribune.com/ice-raids-minnesot...
368,52,2026-01-27-18-23-00 +0000,wsj,Amazon to Shut Down All Amazon Go and Amazon F...,https://www.wsj.com/business/retail/amazon-to-...
192,52,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
58,50,2026-01-28-15-10-00 +0000,wsj,The Bank of Canada kept its policy rate unchan...,https://www.wsj.com/articles/bank-of-canada-st...
265,48,2026-01-27-23-33-54 +0000,latimes,Bari Weiss pushes a digital plan in attempt to...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
